In [5]:
import torch
import torch.nn as nn 
from torch.functional import F
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import xarray as xr
import datashader as ds
import datashader.transfer_functions as tf
import seaborn as sns

from functools import partial
from tqdm.notebook import tqdm
from sconf import Config

import sys
sys.path.append('..')

from classes import MLP, MultiTaskLoss
from utils import non_markovian_midprice

In [7]:
config = Config('config.yaml')

In [8]:
rng = np.random.default_rng(123)

num_paths = config.num_paths
num_timesteps = config.num_timesteps
final_time = config.final_time
initial_price = config.initial_price
decomp_dim = config.decomp_dim
num_episodes = config.num_episodes
batch_size = config.batch_size

dt = final_time/num_timesteps

inital_prices = np.ones((1, num_paths)) * initial_price

permenant_price_impact_func = lambda x: 0.01*x
kernel_function = lambda x,y: (np.abs((x-y)+1e-3)**(0.51-1))
# kernel_function = lambda x,y: (1/(np.sqrt(2*np.pi)))*np.exp(-(x-y)**2)
# kernel_function = lambda x,y: np.exp(-np.abs((x-y)))
K0 = kernel_function(0,0)

nu_vals = rng.uniform(-50, 100,size=(num_timesteps, num_paths))

sigma_true = 0.1

In [9]:
network = MLP(decomp_dim=decomp_dim)
# mtloss = MultiTaskLoss(num_losses=3)
optimizer = torch.optim.Adam(params=[*network.parameters(), 
                                #      *mtloss.parameters()
                                     ],
                             lr=3e-4)

# Constraining Outputs

# Monotonicity

In [25]:
timestamp = 10
time_tensor = torch.FloatTensor([timestamp*dt])

f_vals = network(time_tensor)
lesser_time = torch.FloatTensor([timestamp*rng.uniform(0, timestamp)])
lesser_f_val = network(lesser_time)
greater_time = torch.FloatTensor([timestamp*rng.uniform(timestamp, num_timesteps)])
greater_f_val = network(greater_time)

s_t = f_vals.dot(lesser_f_val)
s_s = lesser_f_val.dot(greater_f_val)

loss = torch.max(torch.zeros_like(s_s),
                  -(s_t-s_s))

loss, s_t, s_s

(tensor(51790.2930, grad_fn=<MaximumBackward0>),
 tensor(2.7330, grad_fn=<DotBackward0>),
 tensor(51793.0273, grad_fn=<DotBackward0>))

# Translation Invariance

In [33]:
timestamp = 10



loss

tensor(5.3773e+12, grad_fn=<PowBackward0>)

tensor(2.7330, grad_fn=<DotBackward0>)